In [ ]:
%%capture
!pip install -qU langchain-chroma

In [ ]:
%%capture
!pip install langchain-community pypdf
!pip install -qU langchain-huggingface
!pip install -U langchain-google-genai
!pip install "unstructured[image]"
!pip install pillow opencv-python
!pip install tqdm
!pip install ddgs
!pip install -U langchain langchain-community langchain-core
!pip install python-docx

**LLM Model Instantiation**

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings # New import

os.environ["GOOGLE_API_KEY"] = ""

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [ ]:
%%capture
!pip install -qU langchain-chroma

In [ ]:
import chromadb
from langchain_chroma import Chroma

cloud_client = chromadb.CloudClient(
    api_key="",
    tenant="",
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="minishlab/potion-base-8M")

vector_store = Chroma(
    client=cloud_client,
    collection_name="rest", # Replace with your actual collection name
    embedding_function=embedding_function # Added embedding function
)

# Tools for R&A Orchestrator Agent

Context Retrieval Tool

In [ ]:
from langchain.tools import tool
from ddgs import DDGS


@tool(response_format="content_and_artifact")
def retrieve_context(query: str, k: int = 3):
    """Retrieve top-k relevant contexts from uploaded research papers."""
    retrieved_docs = vector_store.similarity_search(query, k=k)
    serialized = "\n\n---\n\n".join(
        (
            f"📄 Source: {doc.metadata.get('title', 'Unknown')}\n"
            f"Similarity: {doc.metadata.get('score', 'N/A')}\n"
            f"{doc.page_content}"
        )
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


Summarizer Tool

In [ ]:
from langchain.tools import tool

@tool("chroma_pdf_summarizer", return_direct=True)
def chroma_pdf_summarizer(pdf_name: str) -> str:
    """
    Summarizes an already-embedded research paper from ChromaDB.
    Allows partial match, file name, or full path.
    """

    print(f"📚 Searching ChromaDB for embedded content matching: {pdf_name}")

    vectorstore = vector_store

    # Pull all docs + metadata (Chroma does NOT support regex or partial filter)
    raw = vectorstore.get()

    documents = raw.get("documents", [])
    metadatas = raw.get("metadatas", [])

    if not documents:
        return "⚠️ Vector store is empty. Add data first."

    # --- MATCH BY: partial filename, full filename, or full path ---
    matched_docs = []
    matched_sources = set()

    for doc, meta in zip(documents, metadatas):
        source_meta = meta.get("source", "").lower()
        if pdf_name.lower() in source_meta:      # partial substring match
            matched_docs.append(doc)
            matched_sources.add(meta.get("source", "Unknown"))

    if not matched_docs:
        return (
            f"⚠️ No data found for '{pdf_name}'.\n"
            "💡 Tip: Try passing part of the filename instead of the full path.\n"
        )

    print(f"🧩 Matched {len(matched_docs)} chunks from:")
    for src in matched_sources:
        print("   -", src)

    combined_text = "\n".join(matched_docs)

    prompt = f"""
You are a professional AI research summarizer.
Summarize the following research paper content into structured sections:
- Title
- Abstract
- Introduction
- Methodology
- Results / Findings
- Conclusion

Return clean JSON (no markdown).
Content:
{combined_text}
"""

    print("🧠 Generating structured summary using LLM...")
    response = model.invoke(prompt)

    print("✅ Summary generation complete.")
    return response.content.strip()


Citations Extractor

In [76]:
import re
from pypdf import PdfReader

def extract_citations_from_pdf(pdf_path: str) -> dict:
    """
    Pure Python function that extracts:
    - inline citations
    - reference entries
    - DOIs
    """

    reader = PdfReader(pdf_path)
    full_text = ""

    # Extract all text
    for page in reader.pages:
        page_text = page.extract_text() or ""
        full_text += page_text + "\n"

    # -----------------------------
    # 1️⃣ EXTRACT INLINE CITATIONS
    # -----------------------------

    inline_patterns = [
        r"\(([^()]+?,\s?\d{4}[a-z]?)\)",        # (Smith, 2020)
        r"[A-Z][A-Za-z]+ et al\.,?\s?\d{4}",    # Smith et al., 2021
        r"[A-Z][A-Za-z]+ and [A-Z][A-Za-z]+,?\s?\d{4}",  # Smith and John, 2019
        r"\[\d{1,3}\]"                           # [1], [12], etc.
    ]

    inline_citations = set()
    for pattern in inline_patterns:
        found = re.findall(pattern, full_text)
        for item in found:
            inline_citations.add(item.strip())

    inline_citations = list(inline_citations)

    # -----------------------------
    # 2️⃣ EXTRACT REFERENCE SECTION
    # -----------------------------

    # Locate "References" or "Bibliography"
    match = re.search(r"(References|Bibliography)(.*)", full_text, re.S | re.I)
    if match:
        ref_section = match.group(2)
    else:
        ref_section = full_text  # fallback

    # Split into entries (numbered, bullet, dash)
    ref_entries = re.split(
        r"\n\d{1,3}\.\s|\n\[\d{1,3}\]\s|\n•\s|\n-\s",
        ref_section
    )

    # Clean entries
    ref_entries = [
        entry.strip()
        for entry in ref_entries
        if len(entry.strip()) > 20
    ]

    # -----------------------------
    # 3️⃣ EXTRACT DOIs
    # -----------------------------

    doi_pattern = r"(10\.\d{4,9}/[-._;()/:A-Za-z0-9]+)"
    dois = list(set(re.findall(doi_pattern, full_text)))

    # -----------------------------
    # RETURN STRUCTURED RESULT
    # -----------------------------

    return {
        "inline_citations": inline_citations,
        "reference_entries": ref_entries,
        "dois": dois
    }


In [77]:
from langchain.tools import tool

@tool("extract_references_only", return_direct=True)
def extract_references_only(pdf_path: str) -> list:
    """
    Extract only the reference entries from a PDF using the internal parser.

    Args:
        pdf_path (str): Path to the research paper PDF.

    Returns:
        list: Clean list of reference entries.
    """
    try:
        result = extract_citations_from_pdf(pdf_path)

        reference_list = result.get("reference_entries", [])

        # Make list pretty and safe
        clean_list = [
            ref.replace("\n", " ").strip()
            for ref in reference_list
            if len(ref.strip()) > 5
        ]

        return clean_list

    except Exception as e:
        return {f"Error extracting references: {str(e)}"}


Save Summary to Docx File Tool

In [ ]:
from langchain.tools import tool
from docx import Document
from docx.shared import Pt, Inches
import os
from datetime import datetime

@tool("summary_to_docx", return_direct=True)
def summary_to_docx(summary_text: str, file_name: str = "Research_Summary.docx") -> str:
    """
    Takes a structured summary text and stores it beautifully in a formatted DOCX file.
    Input:
        - summary_text: The summary content (JSON or formatted text)
        - file_name: Optional name for the .docx file (default = Research_Summary.docx)
    Output:
        - Message containing download instructions and file path
    """

    # Create a new Word document
    doc = Document()

    # Add a title
    doc.add_heading("📘 Research Paper Summary", level=0)

    # Add timestamp
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    doc.add_paragraph(f"Generated on: {now}\n")

    # Split summary intelligently (works if it’s JSON or plain text)
    sections = []
    try:
        import json
        parsed = json.loads(summary_text)
        for key, value in parsed.items():
            sections.append((key.title(), value))
    except Exception:
        # fallback — split by section headers
        raw_sections = summary_text.split("\n")
        for line in raw_sections:
            if ":" in line:
                parts = line.split(":", 1)
                sections.append((parts[0].strip().title(), parts[1].strip()))

    # Format and write each section
    for header, body in sections:
        doc.add_heading(header, level=1)
        para = doc.add_paragraph(body)
        para_format = para.paragraph_format
        para_format.space_after = Pt(10)
        para_format.line_spacing = 1.5

    # Save file to local path
    output_dir = "/content/Summaries"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, file_name)
    doc.save(output_path)

    return f"✅ Summary successfully saved as DOCX.\n📄 File path: {output_path}\nYou can now download and share this document."

## R&A Orchestrator Agent

In [83]:
from langchain.agents import create_agent
from langchain_core.language_models.chat_models import BaseChatModel


tools = [summary_to_docx,chroma_pdf_summarizer,retrieve_context,extract_references_only]
prompt = (
          """
              You are an Research and Analysis Orchestrator Agent. Ask the user to tell what he want to do:
              If he want to create a summary of the provided Data -> call the chroma_pdf_summarizer tool
              or If he want to do QnA Research which is RAG Based -> call the retrieve_context tool

              Then if he got the summary , ask if he want to save the summary to docx, if yes then call the summary_to_docx tool.
              If user want to extract the citations, ask him to provide actual path, which is input to the tool -> extract_references_only
              like whatever the path is forexample user says path is "/content/Paper45" then call tool as extract_references_only(/content/paper45)


          """

)

orchestrator  = create_agent(model, tools, system_prompt=prompt)

In [84]:
print("🧠 Interactive Research System Ready!")
print("Type 'exit' to stop.\n")

conversation = []

while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit"]:
        print("👋 Session ended.")
        break

    conversation.append({"role": "user", "content": user_input})

    response = orchestrator.invoke({"messages": conversation})
    ai_message = response["messages"][-1]

    print("\nAI:", ai_message.content, "\n")

    conversation.append({"role": "assistant", "content": ai_message.content})

🧠 Interactive Research System Ready!
Type 'exit' to stop.

You: hi 

AI: Hi there! How can I help you today? Are you looking to create a summary of a document, conduct research with QnA, or extract citations? 

You: extract citations

AI: Great! Please provide the actual path to the document you want to extract citations from. For example, you can say "/content/my_research_paper". 

You: /content/Paper_86-Cardio_Edge_Hardware_Software_Co_design_Implementation.pdf

AI: ["WHO, “Cardiovascular diseases (cvds),” accessed on 19 Feb, 2025. [Online]. Available: https://www.who.int/en/news-room/fact-sheets/de tail/cardiovascular-diseases-(cvds)", "J. M. Bote, J. Recas, F. Rinc ´on, D. Atienza, and R. Hermida, “A mod- ular low-complexity ecg delineation algorithm for real-time embedded systems,”IEEE Journal of Biomedical and Health Informatics, vol. 22, no. 2, pp. 429–441, 2018.", "M. Janveja, R. Parmar, S. Dash, J. Pidanic, and G. Trivedi, “A low- power co-processor to predict ventricular arrh